In [1]:
import os
import pandas as pd
import shutil

import subprocess
from multiprocessing import Pool

In [2]:
base_data_path = '/data/private/SU/bbchip13/brains_on_beats/magnatagatune/'
annotations_filename = base_data_path+'annotations_final_top_50_ver_1.csv'
audio_info = pd.read_csv(annotations_filename)
audio_info.head()

clip_id                                           mp3_path  no voice  \
0        2  f/american_bach_soloists-j_s__bach_solo_cantat...     False   
1        6  f/american_bach_soloists-j_s__bach_solo_cantat...     False   
2       10  f/american_bach_soloists-j_s__bach_solo_cantat...     False   
3       11  f/american_bach_soloists-j_s__bach_solo_cantat...     False   
4       12  f/american_bach_soloists-j_s__bach_solo_cantat...     False   

   harpsichord  sitar  male vocal  vocals  beats  classical  female  ...    \
0        False  False       False   False  False       True   False  ...     
1        False  False       False   False  False       True   False  ...     
2        False  False       False   False  False       True   False  ...     
3        False  False       False   False  False      False   False  ...     
4        False  False       False   False  False       True   False  ...     

    rock  dance  singing  cello  techno  flute   beat   soft  choir  metal  
0  False  False    False  False   False  False  False  False  False  False  
1  False  False    False  False   False  False  False  False  False  False  
2  False  False    False  False   False  False  False  False  False  False  
3  False  False    False  False   False  False  False  False  False  False  
4  False  False    False  False   False  False  False  False  False  False  

[5 rows x 52 columns]

In [3]:
mp3_base_path = base_data_path+'mp3/'
result_base = base_data_path+'split_mp3/'
train_dir = result_base+'train/'
val_dir = result_base+'val/'
test_dir = result_base+'test/'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [4]:
id_list = audio_info['clip_id'].tolist()
file_list = audio_info['mp3_path'].tolist()
for clip_id, filename in zip(id_list, file_list):
    parent = os.path.dirname(filename)
    basename = os.path.basename(filename)
    extension = os.path.splitext(basename)[1]
    
    if (parent >= '0' and parent <= '9') or (parent >= 'a' and parent <= 'b'):
        shutil.copy(mp3_base_path+filename, train_dir+str(clip_id)+extension)
    elif parent == 'c':
        shutil.copy(mp3_base_path+filename, val_dir+str(clip_id)+extension)
    elif parent >= 'd' and parent <= 'f':
        shutil.copy(mp3_base_path+filename, test_dir+str(clip_id)+extension)
    else:
        print('???', filename)

In [5]:
def cvt_wav_by_ffmpeg(input, output):
#    parent = os.path.dirpath(output)
#    os.makedirs(parent, exist_ok=True)
    subprocess.call(['ffmpeg', '-i', input, '-acodec', 'pcm_s16le', '-ar', '16000', 
                     '-threads', '2', '-loglevel', 'error', output])
    return True

In [6]:
train_mp3_list = [train_dir+filename for filename in os.listdir(train_dir) 
                  if filename.endswith('.mp3')]
train_wav_list = [filename.replace('.mp3', '.wav') for filename in train_mp3_list]
with Pool(processes=8) as pool:
    pool.starmap(cvt_wav_by_ffmpeg, zip(train_mp3_list, train_wav_list))

val_mp3_list = [val_dir+filename for filename in os.listdir(val_dir) 
                if filename.endswith('.mp3')]
val_wav_list = [filename.replace('.mp3', '.wav') for filename in val_mp3_list]
with Pool(processes=8) as pool:
    pool.starmap(cvt_wav_by_ffmpeg, zip(val_mp3_list, val_wav_list))
    
test_mp3_list = [test_dir+filename for filename in os.listdir(test_dir) 
                if filename.endswith('.mp3')]
test_wav_list = [filename.replace('.mp3', '.wav') for filename in test_mp3_list]
with Pool(processes=8) as pool:
    pool.starmap(cvt_wav_by_ffmpeg, zip(test_mp3_list, test_wav_list))